<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Lembrando:-Condições-necessárias-de-otimalidade" data-toc-modified-id="Lembrando:-Condições-necessárias-de-otimalidade-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Lembrando: Condições necessárias de otimalidade</a></span></li><li><span><a href="#Exemplo-(exercício-6---p.-27)" data-toc-modified-id="Exemplo-(exercício-6---p.-27)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Exemplo (exercício 6 - p. 27)</a></span></li><li><span><a href="#Exercício-(exemplo-8.1---p.-28)" data-toc-modified-id="Exercício-(exemplo-8.1---p.-28)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Exercício (exemplo 8.1 - p. 28)</a></span></li></ul></div>

# Trabalho computacional

Notas da prof. Soledad, do minicurso de "Controle ótimo com aplicações à Biologia" - *ICTP LATIN AMERICAN SCHOOLIN APPLIED MATHEMATICS*.

- Notas do livro:

*[Lenhart and Workman, 2007] Lenhart, S. and Workman, J. T. (2007). Optimal control applied to biological models. Chapman and Hall/CRC.*

- Importando pacotes

In [1]:
import numpy as np
import pandas as pd
import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

## Lembrando: Condições necessárias de otimalidade

Se $(u*, x*)$ é ótimo, então existe $\lambda : [t_0 , T] \rightarrow R^n$ tal que: 

$$
\begin{cases}
\lambda '(t) & = & -H_x(t, x^*(t), u^*(t), \lambda(t)) \\
\lambda (T) & = & 0 \\
H_u(t, x^*(t), u^*(t), \lambda(t)) & = & 0
\end{cases}
$$

## Exemplo (exercício 6 - p. 27)

Queremos resolver

$$
min \frac{1}{2} \int_0 ^1 (3x^2+u^2) dt \\
sa. x'= x + u, x(1) = 0
$$

- Qual a ideia? Resolver a equação $H_u =  0$ apra achar uma expressão de $u$ (variável algébrica) em função de $x$ e $\lambda$ (variáveis diferenciáveis nas equações)


Sabemos que

$$H(x,u, \lambda) := L + \lambda f$$

No exercício, temos:

$$H= \frac{3}{2}x^2+ \frac{1}{2}u^2 +  \lambda(x + u)$$

Tomando $H_u$:

$$ H_u = u + \lambda =  0 \rightarrow  u = - \lambda (i)$$

Vamos usar $(i)$ em $\lambda'$ para obter a expressão sem $u$:

$$ \lambda' = -H_x = -3x + \lambda =  0 (ii)$$

Assim, obtemos o sistema:

$$
\begin{cases}
x'^* &=&  x - \lambda \\
\lambda'^* &=&  -3x + \lambda\\
\end{cases}
$$

Que pode ser lido de forma matricial: $[x' \lambda'] = [1 -1; -3 1][x \lambda]$, e resolvido analiticamente. Para isso, achamos os autovalores e autovetores da matriz e, derivando, obtemos suas expressões.

OBS1: dado que os dois autovetores formam uma base em $R^2$, a solução é uma combinação linear deles.

OBS2: como é um problema linear, conseguimos resolver de forma analítica, mas vale notar que **não é um problema de condições de contorno**

## Exercício (exemplo 8.1 - p. 28)

$$
min \frac{1}{2} \int _0 ^1 (Ax^2 + Bu^2) dt \\
sa. x'= -\frac{x^2}{2} + Cu \\
x(0) = x_0 > -2
$$

Resolvendo $H_u$ e substituindo $u$ em $x'$, com as condições de otimalidade, temos o sistema:

$$
\begin{cases}
x'^*(t) & = & -\frac{x^2}{2} +\frac{C^2 \lambda}{2B} \\
\lambda'^*(t) & = & -A + \lambda x^* \\
x^*(0) &=& x_0 > -2 \\
\lambda(1) &=& 0
\end{cases}
$$

Vamos resolvê-lo numericamente, em 4 passos:

0. Inicializamos o algoritmo com $u = 0$ e $\delta = 0.1$ (tolerância)
1. Integramos numericamente $x'^*(t)$ de $t_0 \rightarrow T$ para obter $x$
2. Integramos numericamente $\lambda'^*(t)$ de $T \rightarrow t_0$ para obter $\lambda$
3. Atualiza $u$


In [2]:
# Método para integração numérica em (1) e (2)
def euler_1a_ordem(func, y_0, vt):
    """
    Resolve o problema de valor inicial com o método de Euler de 1ª ordem com h = y_n - y_(n-1).
    
    Parameters
    ----------
    func: função lambda(ty) de y' do problema
    y0: valor inicial do problema
    vt: vetor que contém os valores da variável
    
    Returns
    -------
    list: estimação de y nos valores em vt
    """

    t_ant = vt[0]
    y_ant = y_0
    
    est = [y_ant]
    for t in vt[1:]:

        h = t - t_ant
        
        # passo de Taylor
        y = y_ant + func(y_ant, t_ant)*h
        est.append(y)
        
        # atualização dos termos
        y_ant = y
        t_ant = t
    
    return est

* Testando método de Euler (Exercício 2 - p. 23)

In [3]:
t_T = 3
t_0 = 1
x_0 = 2
func = lambda x, t: x + (np.e**t) + (t*x)

vt = np.linspace(t_0, t_T, 50) # 50 pontos
fig = pd.Series(euler_1a_ordem(func, x_0, vt)).iplot(asFigure=True, title='Exercício 2 - Método de Euler<br>f\'(x) = x + e^t + tx')
fig.show()

In [4]:
plotly.offline.plot(fig, filename='graphs/ex_2_euler.html', auto_open=False)

'graphs/ex_2_euler.html'

* Implementação do algoritmo

In [5]:
def exercicio_8_1(A, B, C, n=50):
    """
    Calcula a solução para o problema (P) descrito acima.
    """
    
    # Definições do problema
    L = lambda x, u: A*(x**2) + B*(u**2)
    l_old = 0 # l(1) = 0
    vt = np.linspace(0, 1, n) # t = [0,1]
    
    # Passo 0: Iniciando o algoritmo
    u_old = 0
    e = 0.1
    x_old = -2 # o que entra aqui?
    
    err = 1
    U = lambda l: C*l/2*B
    
    while err > 0.1:
        
        # Passo 1: Integrando x' (t_0, t_T) para obter x
        x_der = lambda x : -(x**2)/2 + C*u_old
        x = sum(euler_1a_ordem(x_der, x_old, vt_1))

        # Passo 2: Integrando l' (t_T, t_0) para obter l
        l_der = lambda l : A - l*x_old
        l = sum(euler_1a_ordem(l_der, l_old, vt_2))  

        # Passo 3: atualização do erro
        u = U(l)
        
        err = max(np.norm(u - u_old)/np.norm(u), 
                  np.norm(x - x_old)/np.norm(x), 
                  np.norm(l - l_old)/np.norm(l))
        
        # Atualização das variáveis do passo seguinte
        x_old = x
        l_old = l
        u_old = u
        
    return L(x, u)